In [36]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import albumentations as A
import torch
import torch.nn as nn
from torchvision.models.segmentation import fcn_resnet50
from albumentations.pytorch import ToTensorV2
from pathlib import Path


> ## 사용법

1. `./data`로 current working directory 변경
2. train 데이터 바꾸고 싶으면 `train_dir = True`, eval 데이터 바꾸고 싶으면 `train_dir = False`
2. 코드 실행
3. `./data/train/images_segmented/...`에 segmented 이미지들 원래랑 똑같은 구조로 저장됨

In [37]:
train_dir = True
target_dir = 'train' if train_dir else 'eval'

In [48]:
segmented_img_dir_path = '/opt/ml/input/data/{}/images_segmented'.format(target_dir)
orig_img_dir_path = '/opt/ml/input/data/{}/images'.format(target_dir)

if not os.path.isdir(segmented_img_dir_path):
    os.mkdir(segmented_img_dir_path)

In [49]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [50]:
model = fcn_resnet50(pretrained=True, progress=False).to(device)
model = model.eval()

In [63]:
def segment_img(img):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = A.Compose([
    ToTensorV2(),
    ])
    
    original_img = img

    img = transform(image=np.array(img))['image']
    img_batch = img.unsqueeze(0).to(device).float()

    output = model(img_batch)['out']
    normalized_masks = torch.nn.functional.softmax(output, dim=1)

    person_mask = normalized_masks[0,15]
    masked_img = person_mask.detach().cpu().numpy()
    
    boundary = 5
    xmax = masked_img.shape[0]
    ymax = masked_img.shape[1]
    
    masked_img = masked_img > 0.9
    temp_masked_img = masked_img.copy()

    for i in range(ymax):
        for j in range(xmax):
            if temp_masked_img[j,i] == True:
                x_left, x_right = max(0,j-boundary), min(xmax-1, j+boundary)
                y_down, y_up = max(0,i-boundary), min(ymax-1, i+boundary)
                masked_img[y_down:y_up, x_left:x_right] = 1

    masked_img = np.expand_dims(masked_img, axis = 2)
    masked_img = np.concatenate((masked_img, masked_img, masked_img), axis=2)
    
    return original_img * masked_img

In [64]:
def train_segment():
    count = 0
    print('start')
    for person_dir in os.listdir(orig_img_dir_path):
        if person_dir.startswith('.'):
            continue
        for fname in os.listdir(Path(orig_img_dir_path) / person_dir):
            if fname.startswith('.'):
                continue
            img_full_path = Path(orig_img_dir_path) / person_dir / fname
            img = Image.open(img_full_path)
            segmented_img = Image.fromarray(segment_img(img))

            new_dir = Path(segmented_img_dir_path) / person_dir
            if not os.path.isdir(new_dir):
                os.makedirs(new_dir)
            new_path = new_dir  / fname
            segmented_img.save(new_path)
        if(count % 100 == 0):
            print(count)
        break
        count += 1
    print('finish')

def eval_segment():
            
    for fname in os.listdir(Path(orig_img_dir_path)):
        if fname.startswith('.'):
            continue
            
        img_full_path = Path(orig_img_dir_path) / fname
        img = Image.open(img_full_path)
        segmented_img = Image.fromarray(segment_img(img))
        
        new_dir = Path(segmented_img_dir_path)
        if not os.path.isdir(new_dir):
            os.makedirs(new_dir)
        
        new_path = new_dir / fname
        segmented_img.save(new_path)


In [65]:
if train_dir:
    train_segment()
else:
    eval_segment()

start
0
finish


In [94]:
torch.cuda.memory

<module 'torch.cuda.memory' from '/opt/conda/lib/python3.8/site-packages/torch/cuda/memory.py'>